### 🆕 Buying Stocks on Margins

- **Purpose**:
    - Create a function that goes through every position in portfolio and calculates if we were to close the position how much our margin would change for both: *Initial and Maintenance*
    - Sort the values as well
    
- **Initial Approach**:✅
    - Using `whatIfOrder(contract, order)` we can retrieve commission and margin impact without actually placing the order.

        - In order to create a `contract` for each position, we can use the Portfolio that we created and extract the Ticker, Quantity, Exchange for Stocks, Ticker, Quantity, Expiry, Exchange, Right, Strike for Options, the contract will either of type `Stock` or `Option`✅

        - We can then `qualify` the contract✅
        - In order to create a order we can create an object of class `Order` , the Order has the following attributes:✅
            - *action*: Sell or Buy
            - *orderType*: MKT, LMT etc
            - *totalQuantity*: Integer
            - *whatIf*: True or False

    - ` whatIfOrder(contract, order)` returns `OrderState` we are interested in the following attributes: *initMarginChange , maintMarginChange* ✅
        - Other attributes include:
        *status, initMarginBefore, maintMarginBefore, equityWithLoanBefore, initMarginChange, maintMarginChange,  equityWithLoanChange, initMarginAfter, maintMarginAfter, equityWithLoanAfter, commission, minCommission, maxCommission, commissionCurrency, warningText, completedTime, completedStatus*


In [1]:
#Import results from portfolio.ipynb
%store -r df_stocks
%store -r df_options
%store -r df_total

In [2]:
import numpy as np
import pandas as pd

#Import ib_insync library
from ib_insync import *

#Only used in interactive environments such as Jupyter Notebooks
util.startLoop()

#Instantiate IB class and use .connect() method on it, if TWS is not running ConnectionRefusedError will be raised
ib = IB()
try:
    print(ib.connect(clientId=0))
except:
    pass

<IB connected to 127.0.0.1:7497 clientId=0>


In [3]:
qualified_stock_contracts = qualified_option_contracts = []
df_stock_orders = df_option_orders = pd.DataFrame()

#Drop totals row from dataframes
df_stocks.drop(df_stocks.tail(1).index,inplace=True)
df_options.drop(df_options.tail(1).index,inplace=True)

#Convert all conIds to Contracts
if df_stocks['ConId'].isnull().values.any() == False:
    #Drop totals row from dataframe
    
    
    #Convert ConId to integers
    df_stocks['ConId'] = df_stocks['ConId'].astype(int)
    
    #Create a Contract using the conId, Ticker, and security type
    df_stock_contracts = df_stocks.apply(lambda x: Contract(conId = x['ConId'], symbol=x['Ticker'], secType='STK'),axis=1)
    
    #Qualify all contracts in both portfolio
    qualified_stock_contracts = ib.qualifyContracts(*df_stock_contracts.values)
    
    #Create Order for each contract
    df_stock_orders = df_stocks.apply(lambda x: Order(action='SELL', orderType='MKT', totalQuantity=x['Qty']),axis=1)
    
if df_options['ConId'].isnull().values.any() == False:
    #Drop totals row from dataframe
    df_options.drop(df_options.tail(1).index,inplace=True)
    display(df_options)
    
    #Convert ConId to integers
    df_options['ConId'] = df_options['ConId'].astype(int)
    
    #Create a Contract using the conId, Ticker, and security type
    df_option_contracts = df_options.apply(lambda x: Contract(conId = x['ConId'], symbol=x['Ticker'], secType='OPT', lastTradeDateOrContractMonth=x['Expiry'], strike=x['Strike'], right=x['Type']),axis=1)
    
    #Qualify all contracts in both portfolio
    qualified_option_contracts = ib.qualifyContracts(*df_option_contracts.values)
    
    #Create Order for each contract
    df_option_orders = df_options.apply(lambda x: Order(action='SELL', orderType='MKT', totalQuantity=x['Qty']),axis=1)



In [4]:
#Iterate over the qualified contracts and orders, pass both in whatif() to get margin details for both stocks and options
stock_initial_margin_changes = []
stock_maint_margin_changes = []
option_initial_margin_changes = []
option_maint_margin_changes = []

for stock_contract, stock_order in zip(qualified_stock_contracts, df_stock_orders.values):
    stock_order_status = (ib.whatIfOrder(stock_contract, stock_order))
    stock_initial_margin_changes.append(float(stock_order_status.initMarginChange))
    stock_maint_margin_changes.append(float(stock_order_status.maintMarginChange))

for option_contract, option_order in zip(qualified_option_contracts, df_option_orders.values):
    option_order_status = (ib.whatIfOrder(option_contract, option_order))
    option_initial_margin_changes.append(float(option_order_status.initMarginChange))
    option_maint_margin_changes.append(float(option_order_status.maintMarginChange))
    
if len(option_initial_margin_changes) == 0 and len(option_maint_margin_changes) == 0  :
    option_initial_margin_changes.append(np.nan)
    option_maint_margin_changes.append(np.nan)
    
if len(stock_initial_margin_changes) == 0 and len(stock_maint_margin_changes) == 0:
    stock_initial_margin_changes.append(np.nan)
    stock_maint_margin_changes.append(np.nan)


In [5]:
#Create columns for margin changes
df_stocks['Initial Margin Change'] = stock_initial_margin_changes
df_stocks['Maintenance Margin Change'] = stock_maint_margin_changes
df_stocks = df_stocks.sort_values(by=['Initial Margin Change', 'Maintenance Margin Change'])

df_options['Initial Margin Change'] = option_initial_margin_changes
df_options['Maintenance Margin Change'] = option_maint_margin_changes
df_options = df_options.sort_values(by=['Initial Margin Change', 'Maintenance Margin Change'])

In [6]:
#Display results
display(df_stocks.style.set_caption("Stocks Margins Portfolio"))

display(df_options.style.set_caption("Options Margins Portfolio"))

,Ticker,Exchange,ConId,Qty,Market Price,Cost Price,P&L,Total Market Value,% Market Value,Initial Margin Change,Maintenance Margin Change
2,QQQ,NASDAQ,320227571,5.000000,335.890015,327.358651,42.660000,1679.450000,52.208068,-422.380000,-422.380000
3,SHOP,NYSE,195014116,1.000000,1106.599976,1175.233457,-68.630000,1106.600000,34.400219,-274.660000,-274.660000
1,FB,NASDAQ,107113386,1.000000,298.500000,309.188357,-10.690000,298.500000,9.279293,-74.720000,-74.720000
0,AAPL,NASDAQ,265598,1.000000,132.289993,126.883257,5.410000,132.290000,4.112421,-33.410000,-33.410000


,Ticker,Exchange,ConId,Qty,Market Price,Cost Price,P&L,Total Market Value,% Market Value,Initial Margin Change,Maintenance Margin Change
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [7]:
ib.disconnect()